# Electricity consumtpion prediction

## Sections
1. Read and prepare datasets
2. Linear regression model
    - Compute correlations between independant variables
    - Define the linear regression model
    - Define the Cross-validation plan
    - Train the model
    - Evaluate the model
    - If necessary, refine the model
3. Random forest model
    - Define the model
    - Define the Cross-validation plan
    - Train the model
    - Evaluate the model
    - If necessary, refine the model
4. Compare performances of both models
5. Interpret the results

### 1. Read and prepare datasets

##### Import libraries

In [73]:
# Import libraries for plotting, reading and wrangling data
library(tidyverse)
library(scales)
library(here)
source(here::here("scripts/Common", "func.R"))

##### Read datasets

In [2]:
# Read electricity datasets: Hourly, Daily, Monthly
ElecHourDF  <-  read_csv(here::here("curated/electricity", "gold_hourly_electricity.csv"),   show_col_types = FALSE)
ElecDayDF   <-  read_csv(here::here("curated/electricity", "gold_daily_electricity.csv"),    show_col_types = FALSE)
ElecMonthDF <-  read_csv(here::here("curated/electricity", "gold_monthly_electricity.csv"),  show_col_types = FALSE) %>%
                mutate(year_month = factor(paste(year, month, sep = "-")))

# Read weather datasets: Hourly, Daily, Monthly
WeatherHourDF  <-   read_csv(here::here("curated/weather", "gold_hourly_weather.csv"),   show_col_types = FALSE)
WeatherDayDF   <-   read_csv(here::here("curated/weather", "gold_daily_weather.csv"),    show_col_types = FALSE)
WeatherMonthDF <-   read_csv(here::here("curated/weather", "gold_monthly_weather.csv"),  show_col_types = FALSE) %>%
                    mutate(year_month = factor(paste(year, month, sep = "-")))

##### Prepare dataframes

In [3]:
# Join hourly weather and electricity data
WEHourDF <- WeatherHourDF %>%
    left_join(ElecHourDF,   by = c("year", "month", "hour"), suffix = c("", "_elec")) %>%
    select(year, month, hour, avg_temp, avg_dewpt_temp, avg_rel_hum_pct, avg_wind_dir, avg_wind_spd, avg_visib, avg_stn_press, avg_hmdx, avg_wind_chill, consumption)
# Join daily weather and electricity data
WEDayDF <- WeatherDayDF %>%
    left_join(ElecDayDF,   by = c("date", "year", "month", "day"), suffix = c("", "_elec")) %>%
    select(date, year, month, day, avg_temp, avg_dewpt_temp, avg_rel_hum_pct, avg_wind_dir, avg_wind_spd, avg_visib, avg_stn_press, avg_hmdx, avg_wind_chill, consumption)
# Join monthly weather and electricity data
WEMonthDF <- WeatherMonthDF %>%
    left_join(ElecMonthDF,   by = c("year_month", "year", "month"), suffix = c("", "_elec")) %>%
    select(year_month, year, month, avg_temp, avg_dewpt_temp, avg_rel_hum_pct, avg_wind_dir, avg_wind_spd, avg_visib, avg_stn_press, avg_hmdx, avg_wind_chill, consumption)

### 2. Linear regression model

##### Compute correlations between independant variables
This will give inputs about interactions that need to be defined in the linear regression model formula

In [4]:
# Define the vector for weather metrics
corrVec <- c("avg_temp", "avg_dewpt_temp", "avg_rel_hum_pct", "avg_visib")

# Create correlation dataframe for hourly data
corrHourDF  <- pairwise_correlations_function(WEHourDF, corrVec, "hour")
# Create correlation dataframe for daily data
corrDayDF   <- pairwise_correlations_function(WEDayDF, corrVec, "day")
# Create correlation dataframe for monthly data
corrMonthDF <- pairwise_correlations_function(WEMonthDF, corrVec, "month")

# Combine all three dataframe together
combinedCorrsDF <- corrHourDF %>%
    union(corrDayDF) %>%
        union(corrMonthDF)

# Display correlation results
head(
    combinedCorrsDF %>%
        pivot_wider(names_from = timeFrame, values_from = correlation)
)


independantVariableOne,independantVariableTwo,hour,day,month
<chr>,<chr>,<dbl>,<dbl>,<dbl>
avg_temp,avg_dewpt_temp,0.888,0.922,0.961
avg_temp,avg_rel_hum_pct,-0.401,-0.324,-0.578
avg_temp,avg_visib,0.394,0.420,0.833
avg_dewpt_temp,avg_rel_hum_pct,0.060,0.067,-0.332
avg_dewpt_temp,avg_visib,0.145,0.174,0.712
avg_rel_hum_pct,avg_visib,-0.588,-0.665,-0.748


##### Define the linear regression model

In [6]:
#=========================
# FORMULAS
#=========================
# define model formula
# Only the independant variables that have a correlation higher than 50% for all time frames are considered for interactions
fmla_linear_regression <- as.formula("consumption ~ avg_temp + avg_dewpt_temp + avg_rel_hum_pct + avg_visib + avg_temp:avg_dewpt_temp + avg_rel_hum_pct:avg_visib")

#=========================
# LINEAR REGRESSION MODELS
#=========================
# Create linear regression models for all three time frames
mdl_linear_regression_hour  <- lm(fmla_linear_regression, data = WEHourDF)
mdl_linear_regression_day   <- lm(fmla_linear_regression, data = WEDayDF)
mdl_linear_regression_month <- lm(fmla_linear_regression, data = WEMonthDF)

# Create matrix containing R-Squared results for all three time frames
ModelsPerfoMatrix <- data.frame(
    timeFrame   = c("hour", "day", "month"),
    rSquared    = c(summary(mdl_linear_regression_hour)$r.squared,      summary(mdl_linear_regression_day)$r.squared,       summary(mdl_linear_regression_month)$r.squared),
    adjRSquared = c(summary(mdl_linear_regression_hour)$adj.r.squared,  summary(mdl_linear_regression_day)$adj.r.squared,   summary(mdl_linear_regression_month)$adj.r.squared)
)
# Display performance results
head(ModelsPerfoMatrix)

,timeFrame,rSquared,adjRSquared
,<chr>,<dbl>,<dbl>
1,hour,0.03247435,0.03246345
2,day,0.25942922,0.25328341
3,month,0.82544078,0.76383164


In [12]:
f_rmse <- function(pDF, pCol, pPredCol) {
    err <- (pDF[pPredCol] - pDF[pCol])[[pPredCol]]
    err2 <- err^2
    rmse <- sqrt(mean(err2))
    return(rmse)
}

In [13]:
f_r_squared <- function(pDF, pCol, pPredCol) {
    err <- pDF[pPredCol] - pDF[pCol]
    rss <- sum(err[, pPredCol]^2)
    #toterr <- pDF[pCol] - mean(pDF[pCol][, pCol])

    toterr <- pDF[pCol][[pCol]] - mean(as.list(pDF[pCol])[[pCol]])
    sstot <- sum(toterr^2)
    r_squared <- 1 - (rss/sstot)

    return(r_squared)  
}

In [14]:
WEMonthDF$pred <- predict(mdl_linear_regression_month, WEMonthDF)
# RMSE
(rmse <- f_rmse(WEMonthDF, "consumption", "pred"))
(sdtemp <- sd(WEMonthDF$consumption))

# R2
(r_squared <- f_r_squared(WEMonthDF, "consumption", "pred"))

[1] 46.04605

[1] 112.5804

[1] 0.8254408

### Cross-Validation

In [72]:
library(vtreat)

In [70]:
set.seed(110)
Rows <- nrow(WEMonthDF)
splitPlan <- kWayCrossValidation(nRows, 3, NULL, NULL)

k <- 3
WEMonthDF$pred.cv <- 0
for(i in 1:k) {
    split <- splitPlan[[i]]
    model <- lm(fmla_linear_regression, data = WEMonthDF[split$train, ])
    WEMonthDF$pred.cv[split$app] <- predict(model, newdata = WEMonthDF[split$app, ])
}

# RMSE
(rmse <- f_rmse(WEMonthDF, "consumption", "pred.cv"))
(sdtemp <- sd(WEMonthDF$consumption))

# R2
(r_squared <- f_r_squared(WEMonthDF, "consumption", "pred.cv"))

[1] 65.50109

[1] 112.5804

[1] 0.646772

In [71]:
WEMonthDF  %>% 
    mutate(diff_pred = abs(pred - consumption), diff_pred.cv = abs(pred.cv - consumption))

year_month,year,month,avg_temp,avg_dewpt_temp,avg_rel_hum_pct,avg_wind_dir,avg_wind_spd,avg_visib,avg_stn_press,avg_hmdx,avg_wind_chill,consumption,pred,pred.cv,diff_pred,diff_pred.cv
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2012-4,2012,4,9.467917,4.6700000,72.71250,18.35833,13.330556,28.73361,101.4942,26.00000,-5.000000,871.43,906.7762,900.2612,35.3461541,28.831220
2012-5,2012,5,12.207661,5.4283602,64.13306,19.77151,14.127688,35.80578,101.8121,26.00000,-5.000000,804.93,853.5761,848.3651,48.6461199,43.435147
2012-6,2012,6,14.148611,8.0479167,67.47500,16.84722,12.858333,30.24722,101.5036,26.00000,-5.000000,708.65,757.4191,747.7381,48.7690677,39.088061
2012-7,2012,7,17.513306,10.6866935,64.81183,16.46237,12.536290,34.40470,101.6377,26.00134,-5.000000,742.74,710.3144,732.3747,32.4256159,10.365263
2012-8,2012,8,18.765860,12.6963710,68.90726,19.10753,12.513441,38.44758,101.5871,26.18817,-5.000000,800.16,704.6996,677.2069,95.4604247,122.953075
2012-9,2012,9,15.228056,10.7530556,75.64306,21.18333,11.200000,39.40250,101.9425,25.99583,-5.000000,720.55,792.3250,802.2413,71.7750030,81.691343
2012-10,2012,10,10.389651,7.3231183,82.15457,15.38306,12.728495,24.50753,101.5207,26.00000,-5.000000,923.40,889.6938,864.5537,33.7062274,58.846265
2012-11,2012,11,7.355139,4.8981944,85.03750,13.90139,13.429167,23.29736,101.4090,26.00000,-4.997222,1073.13,953.3780,930.7573,119.7519851,142.372718
2012-12,2012,12,4.403495,2.4486559,87.36425,14.22177,15.302419,19.40054,101.1006,26.00000,-4.981183,1010.76,1055.6594,1140.6541,44.8994405,129.894134
